# This file move all tfrecord files randomly based on their location to sub folders of train test and validation

In [3]:
import zipfile
import os

In [4]:
def unzip_files_in_directory(directory):
    """
    Unzips all zip files in the given directory.

    Args:
    - directory (str): The path to the directory containing zip files.
    """
    for file in os.listdir(directory):
        if file.endswith('.zip'):
            zip_path = os.path.join(directory, file)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                print(f"Unzipping {file}...")
                zip_ref.extractall(directory)
                print(f"Unzipped {file}.")

In [3]:
unzip_files_in_directory('raw_records')

Unzipping asset-index-2-20240103T071400Z-001.zip...
Unzipped asset-index-2-20240103T071400Z-001.zip.
Unzipping asset-index-2-20240103T071400Z-002.zip...
Unzipped asset-index-2-20240103T071400Z-002.zip.
Unzipping asset-index-2-20240103T071400Z-003.zip...
Unzipped asset-index-2-20240103T071400Z-003.zip.
Unzipping asset-index-2-20240103T071400Z-004.zip...
Unzipped asset-index-2-20240103T071400Z-004.zip.
Unzipping asset-index-2-20240103T071400Z-005.zip...
Unzipped asset-index-2-20240103T071400Z-005.zip.
Unzipping asset-index-20231229T004625Z-001.zip...
Unzipped asset-index-20231229T004625Z-001.zip.
Unzipping asset-index-20231229T004625Z-002.zip...
Unzipped asset-index-20231229T004625Z-002.zip.
Unzipping asset-index-20231229T004625Z-003.zip...
Unzipped asset-index-20231229T004625Z-003.zip.
Unzipping asset-index-20231229T004625Z-004.zip...
Unzipped asset-index-20231229T004625Z-004.zip.
Unzipping asset-index-20231229T004625Z-005.zip...
Unzipped asset-index-20231229T004625Z-005.zip.
Unzipping 

In [18]:
import os
import random
import glob

random.seed(42)

def extract_location_from_filename(filename):
    """
    Extracts the longitude and latitude from the filename.

    Args:
    - filename (str): The filename to parse.

    Returns:
    - (float, float): A tuple of longitude and latitude.
    """
    # Assuming filename format: "year_longitude_latitude_..."
    parts = filename.split('_')
    longitude = float(parts[1])
    latitude = float(parts[2])
    return (longitude, latitude)

def split_files_by_location(directory, train_ratio=0.6, validation_ratio=0.2, test_ratio=0.2):
    """
    Splits files into training, validation, and test sets based on their location.

    Args:
    - directory (str): Directory containing the files.
    - train_ratio (float): Ratio of training set size to total.
    - validation_ratio (float): Ratio of validation set size to total.
    - test_ratio (float): Ratio of test set size to total.

    Returns:
    - train_files, val_files, test_files (tuple): Lists of file paths for training, validation, and test.
    """
    assert train_ratio + validation_ratio + test_ratio == 1, "Ratios must sum up to 1"

    file_paths = glob.glob(os.path.join(directory, '*.tfrecord.gz'))
    file_paths.sort()

    # Group files by location
    location_groups = {}
    for path in file_paths:
        filename = os.path.basename(path)
        location = extract_location_from_filename(filename)
        if location not in location_groups:
            location_groups[location] = []
        location_groups[location].append(path)

    # Split locations into train, validation, and test sets
    locations = list(location_groups.keys())
    random.shuffle(locations)

    train_end = int(len(locations) * train_ratio)
    val_end = train_end + int(len(locations) * validation_ratio)

    train_locations = set(locations[:train_end])
    val_locations = set(locations[train_end:val_end])
    test_locations = set(locations[val_end:])

    # Allocate files to train, validation, or test sets
    train_files, val_files, test_files = [], [], []
    for location, paths in location_groups.items():
        if location in train_locations:
            train_files.extend(paths)
        elif location in val_locations:
            val_files.extend(paths)
        elif location in test_locations:
            test_files.extend(paths)

    return train_files, val_files, test_files

In [19]:
train, val, test = split_files_by_location('raw_records')

In [6]:
import shutil

def move_files_to_subfolders(train_files, val_files, test_files, base_directory):
    """
    Moves training and validation files into respective subfolders.

    Args:
    - train_files (list): List of training file paths.
    - val_files (list): List of validation file paths.
    - test_files (list): List of test file paths
    - base_directory (str): The base directory where the subfolders will be created.
    """

    train_dir = os.path.join(base_directory, 'train')
    val_dir = os.path.join(base_directory, 'val')
    test_dir = os.path.join(base_directory, 'test')

    # Create subfolders if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Move training files
    for file in train_files:
        shutil.move(file, train_dir)
    print(f"Moved {len(train_files)} files to {train_dir}")

    # Move validation files
    for file in val_files:
        shutil.move(file, val_dir)
    print(f"Moved {len(val_files)} files to {val_dir}")

    # Move validation files
    for file in test_files:
        shutil.move(file, test_dir)
    print(f"Moved {len(test_files)} files to {test_dir}")

In [21]:
move_files_to_subfolders(train, val, test, 'raw_records')

Moved 7867 files to raw_records\train
Moved 2636 files to raw_records\val
Moved 2637 files to raw_records\test


# Read the files and concatenate them

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

import torch
from dataloader import read_tfrecord
from tqdm import tqdm
import random


In [2]:
def get_file_paths_shuffled(directory, file_extension='.tfrecord.gz'):
    """
    Get all file paths in the specified directory with the specified file extension.

    Args:
    - directory (str): The directory to search for files.
    - file_extension (str): The file extension to filter by.

    Returns:
    - List[str]: A list of file paths.
    """
    file_paths = []
    for file in os.listdir(directory):
        if file.endswith(file_extension):
            file_path = os.path.join(directory, file)
            file_paths.append(file_path)
    random.seed(42)
    random.shuffle(file_paths)
    return file_paths

def concatenate_and_save_tfrecords(tfrecord_paths, type):
    all_data = []
    all_labels = []
    unread_paths = []
    count = 0
    batch_number = 1
    for path in tqdm(tfrecord_paths):
        try:
            data, label = read_tfrecord(path)  # Read each TFRecord file
        except:
            unread_paths.append(path)
            continue
        all_data.append(data)
        all_labels.append(label)
        count = count+1
        if count == 512:
            # Concatenate all data and labels
            concatenated_data = torch.cat(all_data, dim=0)
            concatenated_labels = torch.cat(all_labels, dim=0)

            torch.save({'data': concatenated_data, 'labels': concatenated_labels}, f'records/{type}/data_batch_{batch_number}.pth')

            # Reset
            all_data = []
            all_labels = []
            count = 0
            
            batch_number = batch_number + 1

    # Concatenate all data and labels
    concatenated_data = torch.cat(all_data, dim=0)
    concatenated_labels = torch.cat(all_labels, dim=0)
    # Save
    torch.save({'data': concatenated_data, 'labels': concatenated_labels}, f'records/{type}/data_batch_{batch_number}.pth')
    return unread_paths

In [4]:
directory = './raw_records/train'
file_paths = get_file_paths_shuffled(directory)
unread_files = concatenate_and_save_tfrecords(file_paths, 'train')
# for file in unread_files:
#     shutil.move(file, './raw_records/train_unread/')

100%|██████████| 6394/6394 [24:16<00:00,  4.39it/s]  


In [5]:
directory = './raw_records/test'
file_paths = get_file_paths_shuffled(directory)
unread_files = concatenate_and_save_tfrecords(file_paths,'test')
# for file in unread_files:
#     shutil.move(file, './raw_records/test_unread/')

100%|██████████| 2131/2131 [07:36<00:00,  4.66it/s]


In [6]:
directory = './raw_records/val'
file_paths = get_file_paths_shuffled(directory)
unread_files = concatenate_and_save_tfrecords(file_paths,'val')
# for file in unread_files:
#     shutil.move(file, './raw_records/val_unread/')

100%|██████████| 2186/2186 [07:50<00:00,  4.64it/s]


# using h5

In [3]:
import h5py
import numpy as np
import tqdm

def concatenate_and_save_tfrecords_as_hdf5(tfrecord_paths, type):
    all_data = []
    all_labels = []
    unread_paths = []
    count = 0
    batch_number = 1
    for path in tqdm.tqdm(tfrecord_paths):
        try:
            data, label = read_tfrecord(path)  # Make sure read_tfrecord function is defined
            data = data.squeeze()
        except:
            unread_paths.append(path)
            continue
        all_data.append(data.numpy())  # Convert data to NumPy array
        all_labels.append(float(label))  # Convert label to float and append
        count += 1
        if count == 512:
            with h5py.File(f'records/{type}/data_batch_{batch_number}.h5', 'w') as h5f:
                h5f.create_dataset('data', data=np.stack(all_data), compression='gzip')
                h5f.create_dataset('labels', data=np.array(all_labels, dtype=np.float32), compression='gzip')
            all_data = []
            all_labels = []
            count = 0
            batch_number += 1

    if all_data:  # Save any remaining data
        with h5py.File(f'records/{type}/data_batch_{batch_number}.h5', 'w') as h5f:
            h5f.create_dataset('data', data=np.stack(all_data), compression='gzip')
            h5f.create_dataset('labels', data=np.array(all_labels, dtype=np.float32), compression='gzip')
    return unread_paths

In [4]:
directory = './raw_records/train'
file_paths = get_file_paths_shuffled(directory)
file_paths
unread_files = concatenate_and_save_tfrecords_as_hdf5(file_paths, 'train')

  9%|▉         | 602/6394 [03:31<24:45,  3.90it/s]   

In [6]:
directory = './raw_records/val'
file_paths = get_file_paths_shuffled(directory)
unread_files = concatenate_and_save_tfrecords_as_hdf5(file_paths,'val')

100%|██████████| 2186/2186 [12:56<00:00,  2.82it/s]  


In [10]:
directory = './raw_records/test'
file_paths = get_file_paths_shuffled(directory)
unread_files = concatenate_and_save_tfrecords_as_hdf5(file_paths,'test')

  0%|          | 0/2131 [00:00<?, ?it/s]

100%|██████████| 2131/2131 [12:01<00:00,  2.96it/s]  
